In [1]:
%run ../ipynb_util_tars.py

In [2]:
%run ../ipynb_load_data.py

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}
Encoded (label2id) label:	 16
Decoded (id2label) label:	 9
9 16 16


In [3]:
torch.set_printoptions(precision=6, sci_mode=False)

In [4]:
"""load pretrained llama"""
import torch
from transformers import AutoTokenizer
from transformers.models.llama.modeling_llama import LlamaForSequenceClassification
from lxt.models.llama import (
    # LlamaForSequenceClassification,
    attnlrp
)

#model_name = f"{CHECKPOINT_PATH}/meta-llama/Meta-Llama-3-8B-ft-zo_up/checkpoint-2200/"
#model_name = f"{CHECKPOINT_PATH}/meta-llama/Meta-Llama-3-8B-fullft-zo_up/checkpoint-1500/"
model_name = f"{CHECKPOINT_PATH}/TinyLlama/TinyLlama_v1.1/checkpoint-100/"
#model_name = f"{CHECKPOINT_PATH}/TinyLlama/TinyLlama_v1.1-ft-zo_up/checkpoint-160/"
#model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = LlamaForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=17,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     token=HF_TOKEN
# )
# model.config.pad_token_id = tokenizer.pad_token_id

# print("Model before AttnLRP rules:", model)

# # apply AttnLRP rules
# print("=" * 40, "\n")
# print("Model after AttnLRP rules:", attnlrp.register(model))


# Use AttnLRP LlamaForSequenceClassification model for comparison
from lxt.models.llama import LlamaForSequenceClassification as LlamaForSequenceClassificationAttnLRP

model_attnlrp = LlamaForSequenceClassificationAttnLRP.from_pretrained(
    model_name,
    num_labels=17,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)
model_attnlrp.eval()
print("=" * 40, "\n")
print("AttnLRP llama rules:", model_attnlrp)

# from peft import PeftModel

# peft_model = PeftModel.from_pretrained(model_attnlrp, model_id=model_name)
# model_attnlrp = peft_model.merge_and_unload()

print("=" * 40, "\n")
print("Model after AttnLRP rules:", attnlrp.register(model_attnlrp, verbose=True))


AttnLRP llama rules: LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LinearEpsilon(in_features=2048, out_features=2048, bias=False)
          (k_proj): LinearEpsilon(in_features=2048, out_features=256, bias=False)
          (v_proj): LinearEpsilon(in_features=2048, out_features=256, bias=False)
          (o_proj): LinearEpsilon(in_features=2048, out_features=2048, bias=False)
          (softmax): Softmax(dim=-1)
          (attn_value_matmul): AttentionValueMatmul()
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): LinearEpsilon(in_features=2048, out_features=5632, bias=False)
          (up_proj): LinearEpsilon(in_features=2048, out_features=5632, bias=False)
          (down_proj): LinearEpsilon(in_features=5632, out_features=2048, bias=False)
     

In [10]:
sample_sentence = "This is about clean energy, and also affordable energy."
#sample_sentence = "Is this about clean energy?"
#sample_sentence = "In terms of regional shares in the OECD area, OECD Europe’s share of consumption is slightly higher than the region’s share of extraction, while the inverse if tme for the OECD America region. The OECD Asia-Oceania region’s share of consumption is the same as its share of extraction. Average income plays a particularly important role. Most of these countries experienced a strong upswing in material extraction starting the early 2000s, although China’s surge began much earlier. By the early 1990s China had overtaken the United States as the world’s largest extractor of material resources."

In [11]:
# Tokenize the sentence
tokenized_output = tokenizer(sample_sentence, return_tensors="pt", add_special_tokens=True)

# Extract token IDs
token_ids = tokenized_output['input_ids'][0].tolist()

# Decode token IDs to readable string
decoded_sentence = tokenizer.decode(token_ids, skip_special_tokens=True)

print("Token IDs:", token_ids)
print("Decoded sentence:", decoded_sentence)

Token IDs: [3, 26, 17, 94, 3029, 477, 585, 8, 67, 4920, 477, 48, 4]
Decoded sentence: this is about clean energy, and also affordable energy.


In [15]:
from lxt.utils import clean_tokens
import lxt.functional as lf

model = model_attnlrp

input_ids = tokenizer(
    sample_sentence,
    return_tensors="pt",
    add_special_tokens=True,
).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits
print("output shape", output_logits.shape)

max_logits, max_indices = torch.max(output_logits[0, :], dim=-1)

print(input_embeds.shape)
max_logits.backward(max_logits)
print(input_embeds.grad.shape)
relevance = input_embeds.grad.float().sum(-1).cpu()[0]

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

print(max_logits, max_indices.item())
print(tokens)
print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
print(relevance)

output shape torch.Size([1, 17])
torch.Size([1, 13, 2048])
torch.Size([1, 13, 2048])
tensor(3.656250, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward0>) 6
['[CLS]', 'this', 'is', 'about', 'clean', 'energy', ',', 'and', 'also', 'affordable', 'energy', '.', '[SEP]']
this is about clean energy, and also affordable energy.
tensor([     0.000000,      0.000000,      0.000000,      0.000000,
             0.110783,     -0.717498,     -1.000000,     -0.000000,
             0.000000,      0.364351,     -0.303981,      0.000000,
             0.000000])


In [8]:
"""Requires `pip install -e ./lxt`"""
import torch
from transformers import AutoTokenizer
from lxt.models.bert import attnlrp, BertForSequenceClassification
from lxt.utils import pdf_heatmap, clean_tokens

def clean_wordpiece_split(tokens):
        """ BERT-specific cleaning. Workaround not working perfect yet."""
        return ["▁" + word.replace("##", "") for word in tokens]

def seq_cls():
    """AttnLRP for BERT sequence classification task."""
    SCIBERT_MODEL = CHECKPOINT_PATH + "/allenai/scibert_scivocab_uncased-ft-lora-zo_up/checkpoint-84/"
    #SCIBERT_MODEL = CHECKPOINT_PATH + "/allenai/scibert_scivocab_uncased-ft-zo_up-lower/checkpoint-240/"
    tokenizer = AutoTokenizer.from_pretrained(
        SCIBERT_MODEL
    )
    model = BertForSequenceClassification.from_pretrained(
        SCIBERT_MODEL,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id
    ).to(torch.device("cuda"))
    model.eval()
    print(model)

    # apply AttnLRP rules
    attnlrp.register(model)
    print(model)

    input_ids = tokenizer(sample_sentence, return_tensors="pt").input_ids.to(torch.device("cuda"))
    inputs_embeds = model.bert.get_input_embeddings()(input_ids)

    logits = model(inputs_embeds=inputs_embeds.requires_grad_()).logits
    print(logits.shape)

    # We explain the sequence label: acceptable or unacceptable
    max_logits, max_indices = torch.max(logits, dim=-1)

    out = model.config.id2label[max_indices.item()]
    print("The label of the sequence is: ", out)

    max_logits.backward(max_logits)

    relevance = inputs_embeds.grad.float().sum(-1).cpu()[0]
    # normalize relevance between [-1, 1] for plotting
    relevance = relevance / relevance.abs().max()

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    tokens = clean_tokens(clean_wordpiece_split(tokens))

    #pdf_heatmap(tokens, relevance, path="./heatmap_seq_cls.pdf", backend="xelatex")
    print(tokens)
    print(relevance)

seq_cls()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNormEpsilon((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_fe

## FinBERT attnlrp on Sentiment classification

In [9]:
finbert = BertForSequenceClassification.from_pretrained(
    "nickmuchi/finbert-tone-finetuned-fintwitter-classification",
    num_labels=3,
).to("cuda")
finbert.eval()
tokenizer = AutoTokenizer.from_pretrained(
    "nickmuchi/finbert-tone-finetuned-fintwitter-classification"
)

sentence = "Citi gives big boost to Deere PT"
print(finbert(input_ids=tokenizer(sentence, return_tensors="pt").input_ids.to("cuda")))
attnlrp.register(finbert)

input_ids = tokenizer(sentence, return_tensors="pt").input_ids.to(torch.device("cuda"))
inputs_embeds = finbert.bert.get_input_embeddings()(input_ids)

logits = finbert(inputs_embeds=inputs_embeds.requires_grad_()).logits
print(logits.shape)

# We explain the sequence label: acceptable or unacceptable
max_logits, max_indices = torch.max(logits, dim=-1)

out = finbert.config.id2label[max_indices.item()]
print("The label of the sequence is: ", out)

max_logits.backward(max_logits)

relevance = inputs_embeds.grad.float().sum(-1).cpu()[0]
# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(clean_wordpiece_split(tokens))

#pdf_heatmap(tokens, relevance, path="./heatmap_seq_cls.pdf", backend="xelatex")
print(tokens)
print(relevance)


SequenceClassifierOutput(loss=None, logits=tensor([[-4.712738,  2.973304, -2.078524]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INIT_MOD_MAP Replacing query with LinearEpsilon
INIT_MOD_MAP Replacing key with LinearEpsilon
INIT_MOD_MAP Replacing value with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing query with LinearEpsilon
INIT_MOD_MAP Replacing key with LinearEpsilon
INIT_MOD_MAP Replacing value with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing query with LinearEpsilon
INIT_MOD_MAP Replacing key with LinearEpsilon
INIT_MOD_MAP Replacing value with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Replacing dense with LinearEpsilon
INIT_MOD_MAP Re